# Deep Convolutional Generative Adversarial Network

In this project we are trying to achieve a Generative Adversarial Network (GAN) model using Deep convolutional technique. In this there are two models such Generator (an artist) and discriminator (an art critic) both the models are set as adversrial to each other to achive the result.

In the first cell we are importing all the libraries that we need for the code to execute.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

physical_devices = tf.config.experimental.list_physical_devices('GPU')      # Enabling GPU
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))  # Checking if GPU is available or not

Num GPUs Available:  1


### Loading dataset

Here we are using the dataset called celeb_A which has 202599 images of celebrities. These images are of 178x218 resolution and 96dpi of pixel density. But for our project we are scaling down the images to 64x64 so that it won't stress the hardware much and runs the program faster.

- For scaling down the image we are using the keras preprocessing fuction

In [2]:
dataset = keras.preprocessing.image_dataset_from_directory(
    directory="C:\\Users\\saika\\Documents\\Jupyter Files\\Project\\archive\\img_align_celeba\\img_align_celeba",
	label_mode=None, image_size=(64, 64), batch_size=32,
    shuffle=True
).map(lambda x: x/255.0)

Found 202599 files belonging to 1 classes.


### Discriminator 

The main goal of the discriminator is to determine whether the given images are fake or not by comparing the images to the genrated images with arrays of 1s.

In [3]:
discriminator = keras.Sequential(
    [
        tf.keras.layers.Input(shape=(64, 64, 3)),
        tf.keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

### Generator

The main goal of the genrator is to generate fake images and to trick the discriminate into making it belive the genrated images are real.

In [4]:
latent_dim = 128
generator = keras.Sequential(
    [
        tf.keras.layers.Input(shape=(latent_dim,)),
        tf.keras.layers.Dense(8*8*128),
        tf.keras.layers.Reshape((8, 8, 128)),
        tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ]                
)

### Discrimator Summary 

In [5]:
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)              0

### Generator Summary

In [6]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 256)      

### Defining Learning rate

Since we are setting two different models we are defining adam optimizer separately.

In [7]:
opt_gen = keras.optimizers.Adam(learning_rate=0.0001)
opt_disc = keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = keras.losses.BinaryCrossentropy()

### Training the model

Here we are training the model in a very generic way. Usually the model is trained to a very extensive range of epoch upto 100+ to see any better result but due to hardware and time restrictions the epoches are redduced to 20 to see faster results. Still with a GPU each epoch took 24~28 minutes.

Once in a while the generated fake images are saved to the defined path so that output can be seen there. At initial stages of the training see lots of losses on the discriminator side, until the adequate traing has been given such as 100 epoch the losses can't be determined.

In [8]:
for epoch in range(20):
    for index, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        fake = generator(random_latent_vectors)

        if index % 100 == 0:
            img = keras.preprocessing.image.array_to_img(fake[0])
            img.save("C:\\Users\\saika\\Documents\\Jupyter Files\\Project\\Generated_images\\gen_img_%03d_%d.png" % (epoch, index))

        with tf.GradientTape() as disc_tape:
            loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
            loss_disc_fake = loss_fn(tf.zeros((batch_size, 1)), discriminator(fake))
            loss_disc = (loss_disc_real + loss_disc_fake)/2

        grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(zip(grads, discriminator.trainable_weights))

        with tf.GradientTape() as gen_tape:
            fake = generator(random_latent_vectors)
            output = discriminator(fake)
            loss_gen = loss_fn(tf.ones(batch_size, 1), output)

        grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(zip(grads, generator.trainable_weights))

100%|██████████| 6332/6332 [27:35<00:00,  3.83it/s]
